## Project: Applying NLP to Major Tasks
#### PROG8245- Machine Learning Programming

| Professor     | Islam Mahmoud               |
|---------------|-----------------------------|
| Email         | imahmoud@conestogac.on.ca   |
| Deadline      | Monday, December 9th, 2024  |
| Group         | Group 10                    |
| Members       | Burman, Jaiv Chaitanya      |
|               | Lad, Khushbu Nileshkumar    |
|               | Madhav, Akshata Ajay        |


### Data Collection

#### Collect a dataset of product reviews. 

##### Rectangular Dataset

- Kaggle Reference : https://www.kaggle.com/datasets/konradb/inside-airbnb-usa/data
- Utilised reviews for : Hawaii
    1. reviews_detailed.csv
    2. listings.csv

##### Import Libraries

In [ ]:
import numpy as np
import pandas as pd 
from scipy import stats
import matplotlib.pyplot as plt
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import PorterStemmer  # For stemming
from nltk.stem import WordNetLemmatizer  # For lemmatization
from deep_translator import GoogleTranslator # For translating review
import string
import nltk
nltk.download('wordnet')
from multiprocessing import Pool
from langdetect import detect, DetectorFactory
from langdetect.lang_detect_exception import LangDetectException
from tqdm import tqdm
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import os


##### Load Dataset classes and methods

In [ ]:
class ReviewsDataset:
    def __init__(self):
        self.listing_file_path = './Dataset/FinalProject/listings.csv'
        self.listing_data = None
        self.reviews_file_path = './Dataset/FinalProject/reviews_detailed.csv'
        self.reviews_data = None
        self.merged_data=None
        self.preprocessed_reviews_file_path = './Dataset/FinalProject/processed_reviews.csv'
        self.is_preprocessed = False
    
    # Loads the data from a CSV file.
    def load_data(self):
        self.listing_data = pd.read_csv(self.listing_file_path)
        print(f"---> Load Listing List \r\n")
        print(f"listings.csv : Data loaded successfully.")
        print(f"Total Records : {self.listing_data.shape[0]} \r\n")

        print(f"---> Load Reviews List \r\n")
        if(os.path.exists(self.preprocessed_reviews_file_path)):
            self.reviews_data = pd.read_csv(self.preprocessed_reviews_file_path)
            print(f"reviews_detailed.csv : Data loaded successfully. ( Preprocessed data )")
            self.is_preprocessed=True
        else:
            self.reviews_data = pd.read_csv(self.reviews_file_path)
            print(f"processed_reviews.csv : Data loaded successfully.")
        print(f"Total Records : {self.reviews_data.shape[0]} \r\n")

    # Print shape of data
    def print_shape(self):
        try:
            print(f"Total Records : Listing : {self.listing_data.shape[0]}")
        except:
            print(f"Total Records : Listing : Not initialized")
        try:
            print(f"Total Records : Review  : {self.reviews_data.shape[0]}")
        except:
            print(f"Total Records : Review : Not initialized")
        try:
            print(f"Total Records : Merged  : {self.merged_data.shape[0]}")
        except:
            print(f"Total Records : Merged : Not initialized")

    
    def merge_data(self):
        # Merge data using common field recepe Id
        self.merged_data = pd.merge(self.listing_data, self.reviews_data, left_on='id', right_on='listing_id')
        print("Data Merged Successfully")
        print(f"Total Records : {self.merged_data.shape[0]} \r\n")



##### The Main function : initialize class object and load data

In [ ]:
if __name__ == "__main__":
    # Create an instance of the ReviewsDataset  class and load data
    review_dataset = ReviewsDataset()
    review_dataset.load_data()


In [ ]:
review_dataset.reviews_data.head(3)

In [ ]:
review_dataset.listing_data.head(3)

##### Null check : Eliminate missing values : listing data

In [ ]:
review_dataset.listing_data.isnull().sum()

In [ ]:
review_dataset.listing_data.dropna(inplace=True)

##### Null check : Eliminate missing values : reviews detail

In [ ]:
review_dataset.reviews_data.isnull().sum()

In [ ]:
review_dataset.reviews_data.dropna(inplace=True)

##### Review shape after null data elimination

In [ ]:
review_dataset.print_shape()

### Annotate the dataset with labels of positive, negative or neutral sentiment, based on collected data.

**Tool** : VADER (Valence Aware Dictionary and sEntiment Reasoner)

In [ ]:
# Initialize the analyzer
analyzer = SentimentIntensityAnalyzer()

# Function to get sentiment scores
def get_sentiment_scores(text):
    sentiment = analyzer.polarity_scores(text)
    result =""
    if(sentiment['compound']<0.5):
        result="Bad"
    elif(sentiment['compound']>0.5):
        result="Good"
    elif(sentiment['compound']==0.5):
        result="Neutral"
    return pd.Series([sentiment['neg'], sentiment['neu'], sentiment['pos'], sentiment['compound'],result ])

# tqdm.pandas()
# review_dataset.reviews_data[['negative', 'neutral', 'positive', 'compound','sentiment']] = review_dataset.reviews_data['comments'].progress_apply(get_sentiment_scores)
# review_dataset.reviews_data.to_csv(review_dataset.preprocessed_reviews_file_path, index=False)


# If previously saved data found, do not reprocess
if(review_dataset.is_preprocessed == False):
    tqdm.pandas()
    review_dataset.reviews_data[['negative', 'neutral', 'positive', 'compound','sentiment']] = review_dataset.reviews_data['comments'].progress_apply(get_sentiment_scores)
else:
    # Show message if data is already loaded
    print("Note : Loaded already saved preprocessed data. Go to next step.")

##### Plot the results

In [ ]:
sentiment_counts = review_dataset.reviews_data['sentiment'].value_counts()
print(sentiment_counts)
# Plotting sentiment distribution without border
plt.figure(figsize=(4, 3))
sentiment_counts.plot(
    kind='bar', 
    color=['green', 'gray', 'red'], 
    alpha=0.7, 
    edgecolor='none'  # Removes border from bars
)
plt.title('Sentiment Distribution of Reviews')
plt.xlabel('Sentiment')
plt.ylabel('Count')
plt.xticks(rotation=0)
plt.tight_layout()

### Preprocessing

1. Perform necessary text preprocessing steps such as tokenization, stop-word removal, stemming/lemmatization, and lowercasing. 
2. Remove any irrelevant columns, handle missing values, and clean text data by removing special characters, stopwords, and performing stemming/lemmatization
3. Handle specific challenges of used text like hashtags, emojis, and slang.

In [ ]:

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
translator = GoogleTranslator(source='auto', target='en')

def PreprocessReviewComment(comment):
    try:
        # Ensure the input is a string
        if not isinstance(comment, str):
            return "",[]

        # Translate to english if other language is detected
        # try:
        #     lang = detect(comment)
        # except LangDetectException:
        #     lang = "unknown"
        translated = comment
        # if lang != "en": 
        #     translated = translator.translate(comment)

        # Tokenization
        word_tokens = word_tokenize(translated) 

        # Remove Stop words and case folding
        stop_words_removed = [word for word in word_tokens if word.lower() not in stop_words]

        # Remove punctuation
        punctuation_removed = [word for word in stop_words_removed if word not in string.punctuation]

        # Lemmatization
        lemmatize_words = [lemmatizer.lemmatize(word) for word in punctuation_removed]

        cleaned_comment = ' '.join(lemmatize_words)
        return cleaned_comment,lemmatize_words
    except:
        return "FAILED",["FAILED"]
    
if(review_dataset.is_preprocessed == False):
    # Initialise plugin for progress
    tqdm.pandas()
    # Preprocess data
    review_dataset.reviews_data[['CleanedComment', 'LemmatizedWords']] = review_dataset.reviews_data['comments'].progress_apply(
        lambda x: pd.Series(PreprocessReviewComment(x))
    )
else:
    # Show message if data is already loaded
    print("Note : Loaded already saved preprocessed data. Go to next step.")

##### Save Preprocessed data in file
- Preprocessing is time taking
- Save and reuse preprocessed data if available
- Translation is taking time so commented (will uncomment and run if time permits)

In [ ]:
# Save intermediate data
if(review_dataset.is_preprocessed == False):
    review_dataset.reviews_data.to_csv(review_dataset.preprocessed_reviews_file_path, index=False)
    print("Data saved successfully at {}")


##### Review processed data

In [ ]:
review_dataset.reviews_data.head(5)

### Feature Extraction and Model Comparison

Explore different feature representation methods such as bag-of-words, TF-IDF, word embeddings (e.g., Word2Vec or GloVe), or contextual embeddings (e.g., BERT or GPT). Experiment with 3 different feature extraction techniques to capture meaningful representations of social media text where the 3 techniques should be of different word embedding categories.

#### TF-IDF

#### Model Building
- Choose a suitable machine learning algorithm (e.g., Naive Bayes, SVM, or neural networks) or deep learning model
- Split the dataset into training and testing sets.
- Train the selected model using the training data, evaluate and record its performance on the training and testing data.

#### Interpretation of results
- Visualize your results
- Compare different feature representation
- Reach a conclusion on which is the best Embedding technique to use for your model

#### Reference
https://www.kaggle.com/datasets/konradb/inside-airbnb-usa/data